In [1]:
import numpy as np
import pandas as pd
from trax import fastmath
from trax import layers as tl
from trax import shapes
from trax.fastmath import numpy as jnp  # For use in defining new layer types.
from trax.shapes import ShapeDtype
from trax.shapes import signature
from trax import supervised as ts
from trax.supervised import training
import trax
from trax.fastmath import jax
from trax.fastmath import jit
import random as rnd

2022-12-13 09:50:11.469165: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-13 09:50:11.868923: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-13 09:50:14.910673: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-13 09:50:14.910852: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-13 09:50:14.910863: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Ca

# Neural network AND gate in Trax example with virtual data

In [2]:
#Create virtual data by adding random noise to AND logic
x1 = jnp.array([0,1])
x2 = jnp.array([1,1])
x3 = jnp.array([1,0])
x4 = jnp.array([0,0])

y1 = jnp.array([0])
y2 = jnp.array([1])
y3 = jnp.array([0])
y4 = jnp.array([0])

X = np.empty((40,2))
X[0:10,:] = np.random.randn(10,2)*0.1 + x1
X[10:20,:] = np.random.randn(10,2)*0.1 + x2
X[20:30,:] = np.random.randn(10,2)*0.1 + x3
X[30:40,:] = np.random.randn(10,2)*0.1 + x4

Y = np.empty((40,1))
Y[0:10,:] = np.random.randn(10,1)*0.1 + y1
Y[10:20,:] = np.random.randn(10,1)*0.1 + y2
Y[20:30,:] = np.random.randn(10,1)*0.1 + y3
Y[30:40,:] = np.random.randn(10,1)*0.1 + y4

In [3]:
#Create data generator that yields all entries
def data_genf(X,Y):
    while True:
        yield (X,Y)

# data_gen = trax.data.inputs.add_loss_weights(data_genf(X,Y))
data_gen = data_genf(X,Y)

In [4]:
model = tl.Serial(
    tl.Dense(2),
    tl.activation_fns.Sigmoid(),
    tl.Dense(1),
    tl.activation_fns.Sigmoid(),
)

In [5]:
train_task = training.TrainTask(
    labeled_data= data_gen,
    loss_layer = tl.BinaryCrossEntropy(), # A cross-entropy loss function
    optimizer = trax.optimizers.Adam(0.01) # The adam optimizer
)
eval_task = training.EvalTask(
    labeled_data = data_gen,  # A labeled data generator
    metrics = [tl.metrics.L2Loss()], # Evaluate with cross-entropy loss and accuracy
    n_eval_batches = 2, # Number of batches to use on each evaluation
)
training_loop = training.Loop( 
    model, # A model to train
    train_task, # A train task
)
# Train with train_steps
training_loop.run(n_steps = 2000)


/home/pd/NNBasics/venv_NNBasics/lib/python3.10/site-packages/jax/_src/lib/xla_bridge.py:553: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(


Will not write evaluation metrics, because output_dir is None.
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save checkpoint as output_dir is None
Did not save chec

In [10]:
print(model(np.array([0,0])))
print(model(np.array([0,1])))
print(model(np.array([1,0])))
print(model(np.array([1,1])))

[0.01015244]
[0.01416808]
[0.01609726]
[0.88716865]


# Extra code snippets (unused)

In [6]:
data_pipeline = trax.data.Serial(
    trax.data.Shuffle(),
    )

streamed_batches = data_pipeline()

In [7]:
#shows computational difference between np and jax
x = np.random.randn(10000,10000).astype(dtype='float32')
def fn(x):
    return x.T + x

%timeit -n5 fn(x)
jax_fn = fn
y = jnp.array(x)
%timeit jax_fn(y).block_until_ready()

1.12 s ± 62.6 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)
680 ms ± 42.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
Y[0,:]

array([-0.02779395])

In [9]:
train_task = training.TrainTask( 
    labeled_data=jnp.vstack(X,Y),      # Use generator (train)
    loss_layer=TripletLoss(),        # Use triplet loss. Don't forget to instantiate this object
    optimizer=trax.optimizers.Adam(learning_rate=0.01),         # Don't forget to add the learning rate parameter
    lr_schedule=trax.lr.warmup_and_rsqrt_decay(400,0.01) # Use Trax multifactor schedule function
)

TypeError: Cannot construct a dtype from an array

In [ ]:
model1 = tl.Serial(
    tl.Embedding(vocab_size=8192, d_feature=256),
    tl.Mean(axis=1),  # Average on axis 1 (length of sentence).
    tl.Dense(2),      # Classify 2 classes.
    tl.LogSoftmax()   # Produce log-probabilities.
)

NameError: name 'tl' is not defined

In [ ]:
model = tl.Serial(
    tl.Dense(2),      # Classify 2 classes.
    tl.LogSoftmax()   # Produce log-probabilities.
)

In [ ]:
def gen1(batch_size,x,y):

    m = x.shape[0]
    m_inds = [*range(m)]
    rnd.shuffle(m_inds)
    ind = 0
    while True:

        X = jnp.empty((batch_size,x.shape[1]))
        Y = jnp.empty((batch_size,y.shape[1]))
        for i in range(batch_size):
            if ind >= m:
                ind = 0
                rnd.shuffle(m_inds)
        
            X[i] = x[m_inds[ind],:]
            Y[i] = y[m_inds[ind],:]

            ind += 1
        
        yield ((X,Y))
        

In [ ]:
X_and = jnp.array([])
Y_and = 

SyntaxError: invalid syntax (225004926.py, line 2)

In [ ]:
train_task = ts.TrainTask(
    
)

NameError: name 'ts' is not defined

In [ ]:
Xt = jnp.array([[1,2],[3,4],[5,6],[7,8]])
Yt = jnp.array([[10],[11],[12],[13]])

In [ ]:
g = gen1(2,Xt,Yt)

In [ ]:
next(g)

(array([[7., 8.],
        [1., 2.]]),
 array([[13.],
        [10.]]))

In [ ]:
help(ts.TrainTask)

Help on class TrainTask in module trax.supervised.training:

class TrainTask(builtins.object)
 |  TrainTask(labeled_data, loss_layer, optimizer, lr_schedule=None, n_steps_per_checkpoint=100, n_steps_per_permanent_checkpoint=None, loss_name=None, sample_batch=None, export_prefix=None)
 |  
 |  A supervised task (labeled data + feedback mechanism) for training.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, labeled_data, loss_layer, optimizer, lr_schedule=None, n_steps_per_checkpoint=100, n_steps_per_permanent_checkpoint=None, loss_name=None, sample_batch=None, export_prefix=None)
 |      Configures a training task.
 |      
 |      Args:
 |        labeled_data: Iterator of batches of labeled data tuples. Each tuple has
 |            1+ data (input value) tensors followed by 1 label (target value)
 |            tensor.  All tensors are NumPy ndarrays or their JAX counterparts.
 |        loss_layer: Layer that computes a scalar value (the "loss") by comparing
 |            model o